In [ ]:
import os
import pandas as pd
import sys
from tqdm import tqdm
import time
sys.path.append('/Users/stevie/repos/lingo_kit_data')

from utils.backend.api import (
    get_term_by_id, update_term_audio, get_term_translation_by_id, update_term_translation_audio,
    get_term_translation_id, get_term_id, get_term_translation_id_list
)

In [ ]:
# df = pd.read_csv('/Users/stevie/repos/lingo_kit_data/dataframes/dataframe_with_audio.csv')
df = pd.read_csv('/Users/stevie/repos/lingo_kit_data/dataframes/dataframe_with_audio_status.csv')
len(df), df.columns

In [ ]:
row = df.iloc[0]

In [ ]:
term_id = get_term_id(row['term_italian'])
term_id

In [ ]:
term_translation_id = get_term_translation_id(
    term_italian_id=term_id,
    part_of_speech=row['part_of_speech'], 
    translation_english=row['translation_english']
)
term_translation_id

In [ ]:
print(get_term_by_id(term_id)['audio_hash_italian'])
# resp = update_term_audio(term_id, '333')
print(get_term_by_id(term_id)['audio_hash_italian'])

In [ ]:
print(get_term_translation_by_id(term_translation_id)['audio_hash_english'])
# resp = update_term_translation_audio(term_translation_id, 'aaa')
print(get_term_translation_by_id(term_translation_id)['audio_hash_english'])

In [ ]:
df['english_audio_hash'].isna().sum(), df['italian_audio_hash'].isna().sum()

In [ ]:
df['status'].value_counts()

In [ ]:
for i, row in tqdm(df.iterrows(), total=len(df)):
    try:
        term_id = get_term_id(row['term_italian'])
        term_translation_id = get_term_translation_id(
            term_italian_id=term_id, 
            part_of_speech=row['part_of_speech'], 
            translation_english=row['translation_english']
        )
        update_term_audio(term_id, row['italian_audio_hash'])
        update_term_translation_audio(term_translation_id, row['english_audio_hash'])
        df.loc[i, 'status'] = 'success'
    except Exception as e:
        print(f"Error processing {row}: {e}")
        df.loc[i, 'status'] = f'error: {e}'
    if i % 10 == 0:
        df.to_csv('/Users/stevie/repos/lingo_kit_data/dataframes/dataframe_with_audio_status.csv', index=False)

In [ ]:
iter_df = df[df['status'] != 'success']
for i, row in tqdm(iter_df.iterrows(), total=len(iter_df)):
    try:
        term_id = get_term_id(row['term_italian'])
        update_term_audio(term_id, row['italian_audio_hash'])

        term_translation_id_list = get_term_translation_id_list(
            term_italian_id=term_id, 
            part_of_speech=row['part_of_speech'], 
            translation_english=row['translation_english']
        )
        for term_translation_id in term_translation_id_list:
            update_term_translation_audio(term_translation_id, row['english_audio_hash'])

        df.loc[i, 'status'] = 'success'
    except Exception as e:
        print(f"Error processing {row}: {e}")
        df.loc[i, 'status'] = f'error: {e}'
    if i % 10 == 0:
        df.to_csv('/Users/stevie/repos/lingo_kit_data/dataframes/dataframe_with_audio_status.csv', index=False)

In [ ]:
df.to_csv('/Users/stevie/repos/lingo_kit_data/dataframes/dataframe_with_audio_status.csv', index=False)

In [ ]:
# Name: 2702, dtype: object: Warning: multiple TermTranslations found for "2514" (pron) -> "myself": 
# [
#     {'id': 2704, 'term': 2514, 'base_lemma': 257, 'translation_english': 'myself', 'part_of_speech': 'pron', 'gender': 'n/a', 'plurality': 's', 'article_type': None, 'article_italian': None, 'preposition': None, 'example_sentence_italian': 'Mi sveglio presto.', 'example_sentence_english': 'I wake up early.', 'audio_hash_english': None, 'notes': 'clitic before the verb; with reflexive verbs'}, 
#     {'id': 2740, 'term': 2514, 'base_lemma': 266, 'translation_english': 'myself', 'part_of_speech': 'pron', 'gender': 'n/a', 'plurality': 'n/a', 'article_type': None, 'article_italian': None, 'preposition': None, 'example_sentence_italian': 'Mi sveglio presto.', 'example_sentence_english': 'I wake up early.', 'audio_hash_english': None, 'notes': 'first-person reflexive; clitic'}
# ]

In [ ]:
# Name: 2761, dtype: object: Warning: multiple TermTranslations found for "2544" (pron) -> "everyone/each one (feminine)": [
#     {'id': 2761, 'term': 2544, 'base_lemma': 269, 'translation_english': 'everyone/each one (feminine)', 'part_of_speech': 'pron', 'gender': 'f', 'plurality': 's', 'article_type': None, 'article_italian': None, 'preposition': None, 'example_sentence_italian': 'Ognuna ha diritto di parlare.', 'example_sentence_english': 'Each one has the right to speak.', 'audio_hash_english': None, 'notes': "feminine form; singular only; often followed by 'di'"}, 
#     {'id': 2763, 'term': 2544, 'base_lemma': 270, 'translation_english': 'everyone/each one (feminine)', 'part_of_speech': 'pron', 'gender': 'f', 'plurality': 's', 'article_type': None, 'article_italian': None, 'preposition': None, 'example_sentence_italian': 'Ognuna porta qualcosa.', 'example_sentence_english': 'Each one brings something.', 'audio_hash_english': None, 'notes': 'feminine form of "ognuno"'}
# ]